## Error handling is boring but this is the Exception

### Vito Minheere

Codebeez meetup 17th of May

Pygrunn 26th of May

Submitted at EuroPython 19th until 21st of July

# What is an Exception?

In [47]:
raise "Exception"

TypeError: exceptions must derive from BaseException

An Exception is an object as many other things in Python. Exception extends the BaseException.

Extend it to create other Exceptions with the same attributes

Internal Python Exceptions should use BaseException, all others should use Exception



BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AssertionError
      ├── AttributeError
      ├── EOFError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError

Built-in errors also have their hierarchy. 

Exception hierarchy can serve an important purpose: 

the user does not have to know all the specific exceptions 

In [48]:
1 / 0

ZeroDivisionError: division by zero

In [49]:
try:
    1 / 0
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e

Arithmetic Error raised


ZeroDivisionError: division by zero

Arithmetic error is higher on the hierarchy than ZeroDivisionError


In [50]:
try:
    1 / 0
    
except Exception as e:
    print("Exception raised")
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e

Exception raised


except Exception is even higher and will be executed first


In [51]:
try:
    thisssss_function_does_not_exist()
    
except Exception as e:
    print("Exception raised")
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e

Exception raised



Be aware that Exception will also catch all other Exceptions derived from it

Python is an interpreted language.
Python uses Exceptions for both reporting errors while running the code and interpreting code. 
If you catch all Exceptions it will also catch NameErrors during interpreting

In [52]:
import sys
try:
    sys.exit()
    
except Exception as e:
    print("SystemExit caught")
    
except BaseException as e:
    print("SystemExit caught by BaseException")  
    
except SystemExit as e:
    print("Raised SystemExit")

SystemExit caught by BaseException


SystemExit is above Exception but extends BaseException

# NAASA, NASA As A Service


#### Launch a rocket via a Python module. 
#### Rocket will be supplied. However we have BYOC - Bring Your Own Crew

### No guarantees, only Exceptions

It is rocket science so a lot of things can go wrong. 

Before the launch you could exit your program and call it off if critical errors arise 

but you can’t just reset the rocket when it is in mid flight. 

In this case we are both the creator of the module and also the user.
We will use a bit of defensive programming to make sure edge cases will be handled and the rocket will not crash. 

## The Rocket

In [56]:
class Rocket:
    def __init__(self, name):
        self.name = name
        self.altitude = 0
        self.crew = []
        self.fuel_tank = FuelTank("Hydrogen", 100)
        self.command_centre = CommandCentre()
        self.engine = Engine(motors=4)
        self.startup_completed = False
        
    def __str__(self):
        return f"Rocket {self.name} has {self.engine.motors} motors and is at an altitude of {self.altitude} meters"
        
rocket = Rocket("Artemis 1")
print(rocket)

Rocket Artemis 1 has 4 motors and is at an altitude of 0 meters


In [57]:
class FuelTank:
    def __init__(self, fuel_type: str, max_volume: int) -> None:
        self.fuel_type = fuel_type
        self.volume = 0
        self.max_volume = max_volume

# Custom Exceptions

In [81]:
 class FuelTankException(Exception):
        pass

raise FuelTankException

FuelTankException: 

In [85]:
class FuelTypeException(FuelTankException):
    
    def __init__(self, fuel_type, allowed_fuel):
        message = f"Invalid fuel type: {fuel_type}. Only '{allowed_fuel}' fuel is allowed."
        super().__init__(message)

class FuelLevelException(FuelTankException):  
    
    def __init__(self, max_volume, volume):
        amount_left_to_fill = max_volume - volume
        message = f"Max amount of fuel to add is {str(amount_left_to_fill)} liters" 
        self.value = amount_left_to_fill
        super().__init__(message)

raise FuelLevelException(10, 7)

FuelLevelException: Max amount of fuel to add is 3 liters

In [86]:
def refuel(self, fuel_type, amount):
    if fuel_type != self.fuel_type:
        raise FuelTypeException(fuel_type)
        
    elif (self.volume + amount) > self.max_volume:
        raise FuelLevelException(self.max_volume, self.volume)
        
    else:
        self.volume += amount
        print(f"Fuel tank refueled. Fuel level is now {self.volume} liter.")

Build a hierarchy in the custom exceptions. FuelException will be caught before FuelTankException when both are in the except clause.



In [87]:
try:
    rocket.fuel_tank.refuel("Diesel", 100)
    
except FuelTankException as e:
    print("FuelTankException")
    print(e)
    raise e
    
except FuelTypeException as e:
    print("FuelTypeException")
    print(e)

FuelTypeException: Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed.

Due to the hierarchy the base FuelTankException can be caught first. 

When you reraise the Exception it shows the FuelTypeException.

End users of our module can put it as the last except block to catch any errors they might have missed.

In [88]:
try:
    rocket.fuel_tank.refuel("Diesel", 100)
    
except FuelLevelException as e:
    print("FuelLevelException")
    print(e)
    raise e
    
except FuelTankException as e:
    print("FuelTankException")
    print(e)
    raise

FuelTypeException: Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed.

## But we "own" both the Rocket and the FuelTank

Since we wrote both Rocket and FuelTank and they only communicate with each other in this scenario. Why not let our system fix itself instead of sending Exceptions and inputs back and forth?

In [ ]:
def refuel(self, fuel_type, amount):
    try:
        self.fuel_tank.refuel(fuel_type, amount)
        
    except FuelTypeException as e:
        print("Switching to correct fuel and retrying")
        self.refuel(self.fuel_tank.fuel_type, amount)
        
    except FuelLevelException as e:
        print("Setting correct fuel amount and retrying")
        self.refuel(fuel_type, e.value)
        
    else:
        print(f"{self.name} refueled.")
        
rocket.refuel("Diesel", 1000)

## Explicit chained exceptions

In [ ]:
class Rocket:
    def __init__(self, name):
        self.name = name
        self.altitude = 0
        self.crew = []
        self.fuel_tank = FuelTank("Hydrogen", 100)
        self.command_centre = CommandCentre()
        self.engine = Engine(motors=4)
        self.startup_completed = False

As we saw earlier the Rocket class had an attribute called startup_completed. 
This could indicate a series of checks that have to be completed before being able to launch.

In [109]:
class Crew:
    def __init__(self, name, ready=False):
        self.name = name
        self.ready = ready
        
    def __str__(self):
        return f"{self.name} is {'ready' if self.ready else 'not ready'}"
        
crew = [Crew("Alice", True), Crew("Bob")]
backup_crew = [Crew("Carol", True)]

for member in crew:
    print(member)

for member in backup_crew:
    print(member)



Alice is ready
Bob is not ready
Carol is ready


In [90]:

class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0].name)
        print("\tThe pilot's name is", crew[1].name)
        print("\tThe mechanic's name is", crew[2].name)
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e

print('Final check procedure')

personnel_check()

Final check procedure
	The captain's name is Alice
	The pilot's name is Bob


RocketNotReadyError: Crew is incomplete

An Exception can be caught and reraised again in the except clause

You can also use raise from to raise a new Exception but keep the old one in the stack trace

In [116]:
crew = [Crew("Alice", True), Crew("Bob")]
backup_crew = [Crew("Carol", True)]

def personnel_check():
    try:
        print("\tThe captain's name is", crew[0].name)
        print("\tThe pilot's name is", crew[1].name)
        print("\tThe mechanic's name is", crew[2].name)
        print("\tThe engineer's name is", crew[3].name)

    except IndexError as e:
        print("Missing crew member, adding backup crew")
        try:
            crew.append(backup_crew.pop())
            personnel_check()
        except IndexError as e:
            raise RocketNotReadyError('Crew is incomplete') from e


try:
    print('Final check procedure')
    personnel_check()
except RocketNotReadyError as f:
    print('General exception: "{}", caused by "{}"'.format(f, f.__cause__))
    raise Exception("Can't launch Rocket today") from f


Final check procedure
	The captain's name is Alice
	The pilot's name is Bob
Missing crew member, adding backup crew
	The captain's name is Alice
	The pilot's name is Bob
	The mechanic's name is Carol
Missing crew member, adding backup crew
General exception: "Crew is incomplete", caused by "pop from empty list"


Exception: Can't launch Rocket today

The raise from can be extended again in another try except

The Exception will travel up the chain to where it wil be caught

See the traceback in the slide

## Creating a checklist

In [119]:
from classes.rocket import Rocket
rocket_2 = Rocket("Artmis 2")

def startup_sequence(self):
    try:
        self.refuel("Diesel", 110) 
        self.test_communication()
        self.check_ignition()
    except StartUpAbort as abort:
        print("Found something wrong, startup halted!")
        raise
    else:
        print("All systems are go!")
        self.startup_completed = True
    finally:
        print("Finished startup check procedure")

rocket_2.startup_sequence()

Switching to correct fuel and retrying
Setting correct fuel amount and retrying
Fuel tank refueled. Fuel level is now 100 liter.
Artmis 2 refueled.
Communication works
Found something wrong, startup halted!
Completed startup check procedure


StartUpAbort: 

# Suppressing errors

In [ ]:
class CommandCentre:
    def __init__(self) -> None:
        self.signal = 100
        
    def generate_transmissions(self):
        messages = [
            "Message 1",
            {"number": 5},
            "Message 2",
            5,
            "Message 3"
        ]
        for message in messages:
            yield message  

In [ ]:
def receive_transmissions():
    centre = CommandCentre()
    transmission_generator = centre.generate_transmissions()
    while True:
        try:
            transmission = next(transmission_generator)
            try:
                transmission = transmission.strip()
            except AttributeError:
                continue
            except TypeError:
                continue
            else:
                print(f"Received: {transmission}")
        except StopIteration:
            print("End of transmission")
            break
        
receive_transmissions()

In [ ]:
from contextlib import suppress

def receive_transmissions():
    centre = CommandCentre()
    transmission_generator = centre.generate_transmissions()    
    while True:
        try:
            transmission = next(transmission_generator)
            with suppress((AttributeError, TypeError)):
                transmission = transmission.strip()
                print(f"Received: {transmission}")
        except StopIteration:
            print("End of transmission")
            break

receive_transmissions()

With suppress uses a ContextManager to handle the Exceptions. The result is the same as the code in the previous slide but it is easier to read and more Pythonic.

Return a context manager that suppresses any of the specified exceptions if they occur in the body of a with statement and then resumes execution with the first statement following the end of the with statement.

## Async Exception Handling

In [ ]:
class MotorError(Exception):
    pass

class IgnitionException(MotorError):
    def __init__(self, motor) -> None:
        self.motor = motor
        message = f"Motor {motor} failed to ignite"
        super().__init__(message)

    def __str__(self):
        return f"Error in engine {self.motor}"

In [ ]:
import asyncio
import random

async def fire_engine(motor):
    print(f"Attempting to fire motor {motor}")
    engine_fires_chance = random.random()
    
    if engine_fires_chance < 0.3:
        raise MotorError(f"Something went wrong in motor {motor}")
        
    elif engine_fires_chance >= 0.8:
        print(f"Motor {motor} failed to fire")
        raise IgnitionException(motor)
        
    print(f"Motor {motor} fired up!")

In [ ]:
async def apply_throttle(motors):
    try:
        async with asyncio.TaskGroup() as tg:
            for x in range(motors):
                tg.create_task(
                    fire_engine(x)
                )
    except* IgnitionException as e:
        print(f"Handling IgnitionErrors: {e.exceptions}")
        print(e.exceptions)
    except* MotorError as e:
        print(f"Handling MotorErrors: {e.exceptions}")
        print(e.exceptions)

In [ ]:
engine = Engine(motors=4)
await apply_throttle(4)